In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical

df = pd.read_csv("/content/Thirukural.csv")

text = df['Verse'].dropna().values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = np.reshape(y, (-1, 1))
y = to_categorical(y, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 300, input_length=max_sequence_len-1))
model.add(LSTM(256))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X, y, epochs=50, batch_size=64, verbose=1)
model.save('thirukural_model.h5')
print("Model saved successfully.")


def generate_text(seed_text, max_sequence_len, tokenizer, model, n_words=7):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    generated_text = seed_text

    for _ in range(n_words):
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)[0]
        predicted_word = tokenizer.index_word[predicted_word_index]

        generated_text += ' ' + predicted_word
        token_list = np.append(token_list, predicted_word_index)[1:]

    return generated_text

seed_text = "பெருங்கடல் வண்டுகள் கழிந்தும்"
generated_text = generate_text(seed_text, max_sequence_len, tokenizer, model, n_words=7)

print(generated_text)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0018 - loss: 8.7369
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0057 - loss: 8.3100
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0067 - loss: 7.9921
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0073 - loss: 7.7027
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0116 - loss: 7.2500
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0259 - loss: 6.5713
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0863 - loss: 5.6842
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2446 - loss: 4.6579
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4624 - loss: 3.6732
Epoch 10/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6729 - loss: 2.6741
Epoch 11/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8022 - loss: 1.8774
Epoch 12/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 

Model saved successfully.
பெருங்கடல் வண்டுகள் கழிந்தும் பெருங்கடல் நீந்துவர் நீந்தார் இறைவன் அடிசேரா தார் தார்


In [ ]:
import pickle


with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
